<a href="https://colab.research.google.com/github/samsenko/quora-duplicates/blob/master/quora_duplicates_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---



**To do:** Reduce number of Epochs to 60; Reduce dropout rate; Figure out how to load model in less time than it takes to train model

---

In [0]:
import spacy
import tensorflow as tf
import numpy as np
import zlib
import glob
import os

from contextlib import suppress
from collections import defaultdict, Counter

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences

from tensorflow.python.keras.layers import Embedding, Input, Activation, Masking, Dense, Dropout, GRU, Bidirectional, BatchNormalization, Lambda, Flatten
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.backend import abs

tf.logging.set_verbosity('WARN')
if 'COLAB_TPU_ADDR' in os.environ:
  TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
else:
  TPU_WORKER = None

Using TensorFlow backend.


In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloaded = drive.CreateFile({'id':'1VHycjtP6NcpmPFyXJxMd-XykMG0dTpzj'}) 
downloaded.GetContentFile('quora_embedded.npz') 

    100% |████████████████████████████████| 993kB 7.8MB/s 

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [0]:
loaded = np.load('quora_embedded.npz')

In [0]:
emb1, emb2, targets = loaded['a'], loaded['b'], loaded['c']

In [0]:
print(emb1.shape, emb2.shape, targets[:50000].shape)

(50000, 30, 300) (50000, 30, 300) (50000,)


In [0]:
def get_siamese_model(input_shape):
    """
        Model architecture
    """
    
    # Define the tensors for the two input questions
    left_input = Input(shape = input_shape)
    right_input = Input(shape = input_shape)
    
    # Recurrent Neural Network
    model = Sequential()
    gru = Bidirectional(
        GRU(
            256,
            dropout=0.5,
            recurrent_dropout=0.5,
            return_sequences=True,
        )
    )
    norm = BatchNormalization()
    dense = Dense(1024)
    flatten = Flatten()
    
    model.add(gru)
    model.add(norm)
    model.add(flatten)
    model.add(dense)
    
    # Generate the encodings (feature vectors) for the two questions
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    
    # Add a customized layer to compute the absolute difference between the encodings
    L1_layer = Lambda(lambda tensors:abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    
    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = Dense(1,activation='sigmoid')(L1_distance)
    
    # Connect the inputs with the outputs
    siamese_net = tf.keras.Model(inputs=[left_input,right_input],outputs=prediction)
    
    #compile model
    siamese_net.summary()
    siamese_net.compile(
      tf.train.AdamOptimizer(learning_rate=0.0001),
      loss='binary_crossentropy',
      metrics=['accuracy'],
    )
    
    # return the model
    return siamese_net

In [0]:
model = get_siamese_model((30, 300))
model = tf.contrib.tpu.keras_to_tpu_model(
  model,
  strategy=tf.contrib.tpu.TPUDistributionStrategy(
    tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_35 (InputLayer)           (None, 30, 300)      0                                            
__________________________________________________________________________________________________
input_36 (InputLayer)           (None, 30, 300)      0                                            
__________________________________________________________________________________________________
sequential_16 (Sequential)      (None, 1024)         16587264    input_35[0][0]                   
                                                                 input_36[0][0]                   
__________________________________________________________________________________________________
lambda_5 (Lambda)               (None, 1024)         0           sequential_16[0][0]              
          

In [0]:
model.fit(
  [emb1, emb2], targets,
  validation_split=0.1,
  epochs=100,
  batch_size=64,
)

Train on 50000 samples, validate on 5000 samples
Epoch 1/100
50000/50000 [==============================] - 349s 7ms/sample - loss: 0.8745 - acc: 0.6005 - val_loss: 0.5738 - val_acc: 0.7100
Epoch 2/100
50000/50000 [==============================] - 73s 1ms/sample - loss: 0.7241 - acc: 0.6502 - val_loss: 0.4699 - val_acc: 0.7590
Epoch 3/100
50000/50000 [==============================] - 74s 1ms/sample - loss: 0.6180 - acc: 0.6905 - val_loss: 0.4067 - val_acc: 0.7938
Epoch 4/100
50000/50000 [==============================] - 74s 1ms/sample - loss: 0.5513 - acc: 0.7207 - val_loss: 0.3785 - val_acc: 0.8112
Epoch 5/100
50000/50000 [==============================] - 73s 1ms/sample - loss: 0.5025 - acc: 0.7469 - val_loss: 0.3165 - val_acc: 0.8416
Epoch 6/100
50000/50000 [==============================] - 74s 1ms/sample - loss: 0.4620 - acc: 0.7693 - val_loss: 0.2827 - val_acc: 0.8640
Epoch 7/100
50000/50000 [==============================] - 73s 1ms/sample - loss: 0.4271 - acc: 0.7899 - val_l

In [0]:
model.save('quora_duplicates_feb19.h5')

In [0]:
from google.colab import auth
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build

In [0]:
drive_service = build('drive', 'v3')

def save_file_to_drive(name, path):
    file_metadata = {
      'name': name,
      'mimeType': 'application/octet-stream'
     }

    media = MediaFileUpload(path, 
                    mimetype='application/octet-stream',
                    resumable=True)

    created = drive_service.files().create(body=file_metadata,
                                   media_body=media,
                                   fields='id').execute()

    print('File ID: {}'.format(created.get('id')))

    return created

save_file_to_drive("quora_duplicates_feb19.h5", "./quora_duplicates_feb19.h5")

File ID: 1v1fTQgqcjM8uUfMkc3ZzxvaMga9S9uy6


{'id': '1v1fTQgqcjM8uUfMkc3ZzxvaMga9S9uy6'}